In [ ]:
from py2neo import Graph, Node, Relationship 
import pandas
import time

In [ ]:
graph = Graph('bolt://localhost:7687', auth=('neo4j', 'password'))
n_nodes = graph.database.primitive_counts['NumberOfNodeIdsInUse']
graph.run("MATCH (b) DETACH DELETE b")

In [ ]:
# Create a CoAuthor graph by creating quanta nodes, then creating relationships between all nodes with relationships
# to that Quanta
query = """
CALL apoc.load.json('file:/Users/timholdsworth/Documents/simple-fake-data.txt') 
YIELD value AS q
MERGE (p:Quanta {name:q.title})
WITH q, p
UNWIND q.authors as author 
MERGE (a:Author {name:author.name})
MERGE (a)-[:CO_CREATOR]->(p)
WITH p,a
MATCH (p)<-[]-(a)
MATCH (b:Author) where (b)-[]->(p) and a <> b
MERGE (a)-[r:COAUTHOR]-(b)
ON CREATE SET r.strength = 1
ON MATCH SET r.strength = r.strength + 1
WITH p DETACH DELETE p
"""

graph.run(query)

In [ ]:
# Divide all relationship scores by two to account for doulbe counting in the first query 
# TODO fix this - its actually setting all the strength values to 0 for some reason
query = """
MATCH (a:Author)-[r:COAUTHOR]-(b:Author)
SET r.strength = r.strength / 2 
"""

graph.run(query)